In [1]:
import numpy as np
import pandas as pd

In [2]:
# ShopX Data

order_data = pd.read_excel("Company ShopX - Order Report.xlsx")
product_weight = pd.read_excel("Company ShopX - Product Weight.xlsx")
pincode_zone = pd.read_excel("Company ShopX - Warehouse&Customer Pin Code and Zone details.xlsx")

#Courier Data
courier_invoice = pd.read_excel("Courier Company - Invoice.xlsx")
courier_rates = pd.read_excel("Courier Company - Rates.xlsx")

In [3]:
# Removing Duplicates

order_data = order_data.drop_duplicates()
product_weight = product_weight.drop_duplicates()
pincode_zone = pincode_zone.drop_duplicates()

In [4]:
# Merging order_report with product_weight
order_data = order_data.merge(product_weight, on='Product Code', how = 'left')

In [5]:
print(order_data.head())

     Order ID   Product Code  Units Ordered  Product Weight (g)
0  2001827036  8904223818706            1.0                 127
1  2001827036  8904223819093            1.0                 150
2  2001827036  8904223819109            1.0                 100
3  2001827036  8904223818430            1.0                 165
4  2001827036  8904223819277            1.0                 350


In [6]:
# Calculating Total_Weight
order_data['Total Weight(g) (ShopX)'] = order_data['Units Ordered'] * order_data['Product Weight (g)']

In [7]:
order_data = order_data.groupby('Order ID')['Total Weight(g) (ShopX)'].sum().reset_index()

In [8]:
order_data['Total Weight(KG) (ShopX)'] = order_data['Total Weight(g) (ShopX)'] / 1000

In [9]:
order_data['Weight Slab(KG) (ShopX)'] = np.ceil(order_data['Total Weight(KG) (ShopX)'] * 2 ) / 2

In [10]:
print(order_data.head())

     Order ID  Total Weight(g) (ShopX)  Total Weight(KG) (ShopX)  \
0  2001806210                    220.0                     0.220   
1  2001806226                    480.0                     0.480   
2  2001806229                    500.0                     0.500   
3  2001806232                   1302.0                     1.302   
4  2001806233                    245.0                     0.245   

   Weight Slab(KG) (ShopX)  
0                      0.5  
1                      0.5  
2                      0.5  
3                      1.5  
4                      0.5  


In [11]:
# Merging

merged_data = pd.merge(order_data,courier_invoice, on='Order ID', how='left')

In [12]:
print(merged_data.head())

     Order ID  Total Weight(g) (ShopX)  Total Weight(KG) (ShopX)  \
0  2001806210                    220.0                     0.220   
1  2001806226                    480.0                     0.480   
2  2001806229                    500.0                     0.500   
3  2001806232                   1302.0                     1.302   
4  2001806233                    245.0                     0.245   

   Weight Slab(KG) (ShopX)  AWB Code (Airway Bill Number)   Chargeable Weight  \
0                      0.5                   1091117221940               2.92   
1                      0.5                   1091117222065               0.68   
2                      0.5                   1091117222080               0.71   
3                      1.5                   1091117222124               1.30   
4                      0.5                   1091117222135               0.78   

   Store House Pincode  Customer Area Code Delivery Zone     Freight Type  \
0               121003     

In [13]:
final_data = pd.merge(merged_data, pincode_zone, on=['Store House Pincode','Customer Area Code'],how='left', suffixes=('', '_ShopX'))

In [14]:
print(final_data.head())

     Order ID  Total Weight(g) (ShopX)  Total Weight(KG) (ShopX)  \
0  2001806210                    220.0                     0.220   
1  2001806226                    480.0                     0.480   
2  2001806229                    500.0                     0.500   
3  2001806232                   1302.0                     1.302   
4  2001806233                    245.0                     0.245   

   Weight Slab(KG) (ShopX)  AWB Code (Airway Bill Number)   Chargeable Weight  \
0                      0.5                   1091117221940               2.92   
1                      0.5                   1091117222065               0.68   
2                      0.5                   1091117222080               0.71   
3                      1.5                   1091117222124               1.30   
4                      0.5                   1091117222135               0.78   

   Store House Pincode  Customer Area Code Delivery Zone     Freight Type  \
0               121003     

In [15]:
def calculated_expected_charges(row):
    # zone_prefix = [a,b,c,d,e]
    zone_prefix = row['Delivery Zone_ShopX'].lower()

    # taking fixed & additional column charges directly
    fwd_fixed_col = f'fwd_{zone_prefix}_fixed'
    fwd_additional_col = f'fwd_{zone_prefix}_additional'
    rto_fixed_col = f'rto_{zone_prefix}_fixed'
    rto_additional_col = f'rto_{zone_prefix}_additional'

    # Ensure that the rquired column exist in courier_rates data
    if all(col in courier_rates.columns for col in [fwd_fixed_col, fwd_additional_col, rto_fixed_col, rto_additional_col]):

        if row['Freight Type'] == 'Forward charges':
            fixed_charges = courier_rates[fwd_fixed_col].values[0]
            additional_charges = courier_rates[fwd_additional_col].values[0] * (row['Weight Slab(KG) (ShopX)'] - 0.5) / 0.5
            return fixed_charges + additional_charges

        elif row['Freight Type'] == 'Forward and RTO charges':
            fwd_fixed_charges = courier_rates[fwd_fixed_col].values[0]
            rto_fixed_charges = courier_rates[rto_fixed_col].values[0]
            fwd_additional_charges = courier_rates[fwd_additional_col].values[0] * (row['Weight Slab(KG) (ShopX)'] - 0.5) / 0.5
            rto_additional_charges = courier_rates[rto_additional_col].values[0] * (row['Weight Slab(KG) (ShopX)'] - 0.5)  / 0.5
            return fwd_fixed_charges + rto_fixed_charges + fwd_additional_charges + rto_additional_charges

    else:
    # If no matching rate is found, return NaN or 0
        return np.nan

In [16]:
final_data['Expected Charges (ShopX)'] = final_data.apply(lambda row: calculated_expected_charges(row), axis=1)

In [17]:
final_data['Difference in Charges (Rs.)'] = final_data['Expected Charges (ShopX)'] - final_data['Total Amount (Rs.)']

In [18]:
print(final_data.head())

     Order ID  Total Weight(g) (ShopX)  Total Weight(KG) (ShopX)  \
0  2001806210                    220.0                     0.220   
1  2001806226                    480.0                     0.480   
2  2001806229                    500.0                     0.500   
3  2001806232                   1302.0                     1.302   
4  2001806233                    245.0                     0.245   

   Weight Slab(KG) (ShopX)  AWB Code (Airway Bill Number)   Chargeable Weight  \
0                      0.5                   1091117221940               2.92   
1                      0.5                   1091117222065               0.68   
2                      0.5                   1091117222080               0.71   
3                      1.5                   1091117222124               1.30   
4                      0.5                   1091117222135               0.78   

   Store House Pincode  Customer Area Code Delivery Zone     Freight Type  \
0               121003     

OUTPUT DATA 1

In [19]:
final_data.columns

Index(['Order ID', 'Total Weight(g) (ShopX)', 'Total Weight(KG) (ShopX)',
       'Weight Slab(KG) (ShopX)', 'AWB Code (Airway Bill Number) ',
       'Chargeable Weight', 'Store House Pincode', 'Customer Area Code',
       'Delivery Zone', 'Freight Type', 'Total Amount (Rs.)',
       'Delivery Zone_ShopX', 'Expected Charges (ShopX)',
       'Difference in Charges (Rs.)'],
      dtype='object')

In [20]:
output_data_1 = final_data[['Order ID','AWB Code (Airway Bill Number) ','Total Weight(g) (ShopX)', 'Total Weight(KG) (ShopX)',
       'Weight Slab(KG) (ShopX)','Chargeable Weight','Delivery Zone_ShopX','Delivery Zone','Expected Charges (ShopX)','Total Amount (Rs.)','Difference in Charges (Rs.)']]

In [21]:
print(output_data_1.head())

     Order ID  AWB Code (Airway Bill Number)   Total Weight(g) (ShopX)  \
0  2001806210                   1091117221940                    220.0   
1  2001806226                   1091117222065                    480.0   
2  2001806229                   1091117222080                    500.0   
3  2001806232                   1091117222124                   1302.0   
4  2001806233                   1091117222135                    245.0   

   Total Weight(KG) (ShopX)  Weight Slab(KG) (ShopX)  Chargeable Weight  \
0                     0.220                      0.5               2.92   
1                     0.480                      0.5               0.68   
2                     0.500                      0.5               0.71   
3                     1.302                      1.5               1.30   
4                     0.245                      0.5               0.78   

  Delivery Zone_ShopX Delivery Zone  Expected Charges (ShopX)  \
0                   b             b    

OUTPUT 2

In [22]:
correctly_charged_orders = final_data[final_data['Difference in Charges (Rs.)'] == 0]

In [23]:
overcharged_orders = final_data[final_data['Difference in Charges (Rs.)'] < 0]

In [24]:
undercharged_orders = final_data[final_data['Difference in Charges (Rs.)'] > 0]

In [25]:
correctly_charged_count = correctly_charged_orders.shape[0]
correctly_charged_total_invoice = correctly_charged_orders['Total Amount (Rs.)'].sum()

In [26]:
overcharged_count = overcharged_orders.shape[0]
overcharged_total_invoice = overcharged_orders['Total Amount (Rs.)'].sum()

In [27]:
undercharged_count = undercharged_orders.shape[0]
undercharged_total_invoice = undercharged_orders['Total Amount (Rs.)'].sum()

In [28]:
output_data_2 = {
    'Charge Type': ['Correctly Charged Orders', 'Overcharged Orders', 'Undercharged Orders'],
    'Order Count': [correctly_charged_count, overcharged_count, undercharged_count],
    'Total Invoice Amount (Rs.)': [correctly_charged_total_invoice, overcharged_total_invoice, undercharged_total_invoice]
}

In [29]:
output_data_2_df = pd.DataFrame(output_data_2)

In [30]:
print(output_data_2_df)

                Charge Type  Order Count  Total Invoice Amount (Rs.)
0  Correctly Charged Orders           15                      1207.3
1        Overcharged Orders           85                      8982.1
2       Undercharged Orders           24                      3458.8


Extracting Output into Excel

In [31]:
with pd.ExcelWriter("Project_Logistics.xlsx") as writer:
    output_data_1.to_excel(writer, sheet_name="Output 1",index=False)
    output_data_2_df.to_excel(writer, sheet_name="Output 2", index=False)